In [2]:
from triplet_preparation import tuples_from_table
from model_architecture import triplet_network_model

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

tf.__version__, tf.keras.__version__

('2.1.0', '2.2.4-tf')

In [3]:
'''Test Dataset'''

file_path = os.path.abspath('../data/samples/lichess_db_standard_rated_2013-01-tuples.h5')
triplets = tuples_from_table(file_path, "tuples_0", tuple_indices=[0,1,6])

train_triplets, test_triplets = train_test_split(triplets, test_size=0.2, random_state=42)
train_triplets.shape, test_triplets.shape

<KeysViewHDF5 ['tuples_0', 'tuples_1']>


((72289, 3, 773), (18073, 3, 773))

In [4]:
'''Initialize and use triplet network'''
input_shape = ((10000,773))
network = triplet_network_model(input_shape, 10)
optimizer = keras.optimizers.Adam(lr = 0.00006)
network_train.compile(loss=None,optimizer=optimizer)
network_train.summary()

NameError: name 'Adam' is not defined

In [4]:
'''Define Model'''

def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [9]:
'''Create Checkpoints during training'''

checkpoint_path = "checkpoints/test_cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
history = model.fit(train_images, 
                    train_labels,  
                    epochs=10,
                    validation_data=(test_images,test_labels),
                    callbacks=[cp_callback])  # Pass callback to training


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 960/1000 [===========================>..] - ETA: 0s - loss: 1.2638 - accuracy: 0.6448
Epoch 00001: saving model to checkpoints/test_cp.ckpt
1000/1000 [==============================] - 1s 653us/sample - loss: 1.2335 - accuracy: 0.6530 - val_loss: 0.7204 - val_accuracy: 0.7850
Epoch 2/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.4300 - accuracy: 0.8792
Epoch 00002: saving model to checkpoints/test_cp.ckpt
1000/1000 [==============================] - 0s 296us/sample - loss: 0.4227 - accuracy: 0.8810 - val_loss: 0.5260 - val_accuracy: 0.8370
Epoch 3/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.3052 - accuracy: 0.9167
Epoch 00003: saving model to checkpoints/test_cp.ckpt
1000/1000 [==============================] - 0s 332us/sample - loss: 0.2916 - accuracy: 0.9230 - val_loss: 0.4594 - val_accuracy: 0.8510
Epoch 4/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.1980 - accu

In [10]:
'''Loads the weights from checkpoint path'''
model = create_model()
model.load_weights(checkpoint_path)

In [11]:
'''Save the complete model after training'''
model.save('model/test_model', save_format='tf') # or save_format='h5'

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model/test_model/assets


In [12]:
'''Load model from saved state'''
model = tf.keras.models.load_model('model/test_model')
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [13]:
'''Use loaded model for inference'''
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))
print(model.predict(test_images).shape)

1000/1000 - 0s - loss: 0.3975 - accuracy: 0.8780
Restored model, accuracy: 87.80%
(1000, 10)
